# AI Гид (solution by "Улыбка Мона Лизы")

## Сбор и подготовка данных

### Импорты

In [ ]:
!pip install -q langchain-mistralai

In [ ]:
from google.colab import userdata
MISTRAL_API_KEY = userdata.get('MISTRAL_API_KEY')

In [ ]:
import json
import csv
import os
import time
from tqdm import tqdm
from pathlib import Path
from typing import List, Dict, Optional
import requests
from langchain_mistralai import ChatMistralAI

### Константы

In [ ]:
API_URL = "https://ru.wikivoyage.org/w/api.php"
HEADERS = {
    "User-Agent": "TravelAIBot/1.0"
}

In [ ]:
DATA_DIR = Path("dataset")
RAW_FILE = DATA_DIR / "wikivoyage_ru_raw.jsonl"
CLEAN_FILE = DATA_DIR / "wikivoyage_ru_clean.jsonl"
FINAL_FILE = DATA_DIR / "wikivoyage_ru_final.jsonl"
CSV_FILE = DATA_DIR / "wikivoyage_ru_final.csv"
DATA_DIR.mkdir(exist_ok=True)

In [ ]:
LLM = ChatMistralAI(
    model="mistral-small-latest",
    temperature=0.0,
    mistral_api_key=MISTRAL_API_KEY
)

In [ ]:
PROMPT = """Извлеки информацию ТОЛЬКО из переданного вики-текста статьи.
НИЧЕГО НЕ ПРИДУМЫВАЙ и не добавляй из своих знаний.

Если в тексте прямо и явно НЕ написано — ставь null.

Верни ТОЛЬКО валидный JSON без каких-либо пояснений:

{
  "country": "страна (или null)",
  "city": "точное название города/региона/объекта (или null)",
  "description": "краткое описание в 2–4 предложения на русском языке",
  "attractions": ["названия достопримечательностей из текста", "..."],
  "tips_for_traveler": "одним абзацем только то, что написано в статье про транспорт, визы, безопасность, деньги, здоровье и т.п. (или null, если таких сведений нет)"
}
Обрабатывай только основной текст, игнорируй шаблоны, категории и служебную информацию.

Вики-текст статьи:
"""

### Вспомогательные функции

In [ ]:
def log(*args):
    print("[*]", *args)

In [ ]:
# Проверка для фильтрации пустых страниц
def is_redirect(wikitext: str):
    return wikitext.strip().upper().startswith(("#REDIRECT", "#ПЕРЕНАПРАВЛЕНИЕ"))

In [ ]:
def append_jsonl(item: Dict, path: Path):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [ ]:
def api_get(params: dict):
    r = requests.get(API_URL, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()

In [ ]:
# Получение содержимого страницы
def fetch_wikitext(title: str):
    data = api_get({
        "action": "query",
        "format": "json",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "titles": title
    })
    page = next(iter(data["query"]["pages"].values()))
    return page["revisions"][0]["slots"]["main"]["*"] if "revisions" in page else None

In [ ]:
# Проверка существования страницы в файле результата
def is_already_saved(title: str, path: str):
    if not os.path.exists(path):
        return False
    with open(path, "r", encoding="utf-8") as f:
        return any(json.loads(line).get("title") == title for line in f)

In [ ]:
# Получение списка страниц
def get_all_pages():
    pages = []
    cont = None
    print("Получаем список всех страниц...")
    while True:
        params = {
            "action": "query",
            "format": "json",
            "list": "allpages",
            "aplimit": "max",
            "apnamespace": "0"
        }
        if cont:
            params["apcontinue"] = cont

        r = requests.get(API_URL, params=params, headers=HEADERS, timeout=30)
        r.raise_for_status()
        data = r.json()
        pages.extend(data["query"]["allpages"])

        cont = data.get("continue", {}).get("apcontinue")
        if not cont:
            break
        time.sleep(0.2)

    print(f"Найдено страниц: {len(pages)}")
    return [page['title'] for page in pages]

### Сохранение списка страниц

In [ ]:
titles_file = "dataset/titles_cache.txt"
if os.path.exists(titles_file):
    titles = [line.strip() for line in open(titles_file, "r", encoding="utf-8")]
    print("Список существует локально")
else:
    titles = get_all_pages()
    with open(titles_file, "w", encoding="utf-8") as f:
        for t in titles:
            f.write(t + "\n")
    print(f"Список сохранён в {titles_file}")

Список существует локально


### Скачивание содержимого

In [ ]:
def download_wikitext(titles: list, output_file: Path):

    print(f"Скачивание содержимого {len(titles)} статей...")
    saved = 0

    # Получаем список уже обработанных заголовков
    processed_titles = set()
    if os.path.exists(output_file):
        with open(output_file, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    processed_titles.add(data["title"])
                except:
                    continue

    # Фильтруем только те заголовки, которые еще не обработаны
    titles_to_process = [title for title in titles if title not in processed_titles]

    if not titles_to_process:
        print(f"Все статьи уже скачаны в {output_file}")
        return

    with open(output_file, "a", encoding="utf-8") as f:
        with tqdm(titles_to_process, desc="Скачивание") as pbar:
            for title in pbar:
                text = fetch_wikitext(title)
                if text:
                    record = {
                        "title": title,
                        "url": f"https://ru.wikivoyage.org/wiki/{title.replace(' ', '_')}",
                        "wikitext": text
                    }
                    f.write(json.dumps(record, ensure_ascii=False) + "\n")
                    saved += 1

                time.sleep(0.33)

    print(f"Готово: {output_file} (+{saved} новых)")

In [ ]:
download_wikitext(titles, RAW_FILE)

Скачивание содержимого 7887 статей...
Все статьи уже скачаны в dataset/wikivoyage_ru_raw.jsonl


### Очистка данных

In [ ]:
if RAW_FILE.exists():
    print(f"Содержимое файла {RAW_FILE.name} (первые 5 строк):")
    with open(RAW_FILE, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            print(line.strip())
else:
    print(f"Файл {RAW_FILE.name} не найден. Убедитесь, что он был создан на предыдущих этапах.")

Содержимое файла wikivoyage_ru_raw.jsonl (первые 5 строк):
{"title": "7+2", "url": "https://ru.wikivoyage.org/wiki/7+2", "wikitext": "#REDIRECT [[Project:Географическая иерархия#Деление на географические единицы]]"}
{"title": "7 2", "url": "https://ru.wikivoyage.org/wiki/7_2", "wikitext": "#REDIRECT [[Project:Географическая_иерархия#Деление на географические единицы]]"}
{"title": "Main Page", "url": "https://ru.wikivoyage.org/wiki/Main_Page", "wikitext": "#перенаправление [[Заглавная страница]]"}
{"title": "WLE", "url": "https://ru.wikivoyage.org/wiki/WLE", "wikitext": "#REDIRECT[[Wikivoyage:Вики любит Землю]]"}
{"title": "WLM", "url": "https://ru.wikivoyage.org/wiki/WLM", "wikitext": "#REDIRECT[[Wikivoyage:Вики любит памятники]]"}


In [ ]:
# Удаление редиректов
def remove_redirects():
    if CLEAN_FILE.exists():
        log(f"{CLEAN_FILE.name} уже существует, пропускаем очистку")
        return

    log("Удаляем редиректы...")
    clean_count = redirect_count = 0

    with open(RAW_FILE, "r", encoding="utf-8") as src, \
         open(CLEAN_FILE, "w", encoding="utf-8") as dst:
        for line in src:
            data = json.loads(line)
            if is_redirect(data["wikitext"]):
                redirect_count += 1
            else:
                dst.write(line)
                clean_count += 1

    log(f"""Оставлено: {clean_count}, удалено строк:
    {redirect_count}. Итоговый файл - {CLEAN_FILE}""")


In [ ]:
remove_redirects()

[*] wikivoyage_ru_clean.jsonl уже существует, пропускаем очистку


### Извлечение признаков для табличных данных (Mistral API)

Для итоговой структуры данных для RAG выбраны признаки:

- title/заголовок страницы
- country/страна
- city/город
- attractions/интересные места
- description/описание
- tips for traveler/советы путешественникам

и служебное поле url

In [ ]:
# Проверка обработанных строк
def get_processed_titles():
    titles = set()
    if FINAL_FILE.exists():
        with open(FINAL_FILE, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    titles.add(json.loads(line)["title"])
                except:
                    pass
    return titles

In [ ]:
# Извлечение признаков через Mistral
def extract_features():
    from tqdm import tqdm

    # Читаем все title, которые уже есть в результате
    processed_titles = set()
    if open(FINAL_FILE, "a+", encoding="utf-8").tell() > 0:  # если файл не пустой
        with open(FINAL_FILE, "r", encoding="utf-8") as f:
            for line in f:
                if line.strip():
                    data = json.loads(line)
                    processed_titles.add(data.get("title"))

    print(f"Уже обработано статей: {len(processed_titles)}")

    # Открываем исходный файл и ищем недостающие
    missing = []
    with open(CLEAN_FILE, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            if data["title"] not in processed_titles:
                missing.append(data)

    print(f"Найдено недостающих/ошибочных: {len(missing)}")

    if not missing:
        print("Всё уже обработано! Можно делать CSV.")
    else:
        print("Обработка недостающих через Mistral...\n")

        with open(FINAL_FILE, "a", encoding="utf-8") as outfile:
            with tqdm(missing, desc="Обработка статей", unit="стран") as pbar:
                for item in pbar:
                    title = item["title"]
                    wikitext = item["wikitext"]

                    # pbar.set_postfix({"статья": title[:30] + "..." if len(title) > 30 else title})

                    for attempt in range(5):
                        try:
                            response = LLM.invoke(
                                [
                                    ("system", PROMPT),
                                    ("user", wikitext[:28000])
                                ],
                                response_format={"type": "json_object"}
                            )

                            result = json.loads(response.content)
                            result["title"] = title
                            result["url"] = item["url"]

                            outfile.write(json.dumps(result, ensure_ascii=False) + "\n")
                            outfile.flush()
                            break

                        except Exception as e:
                            # if "429" in str(e) or "rate limit" in str(e).lower():
                            #     wait = 5 * (attempt + 1)
                            #     pbar.write(f" → 429, ждём {wait} сек...")
                            #     time.sleep(wait)
                            # else:
                            # pbar.write(f"Ошибка для '{title}': {e}")
                            break
                    else:
                        pbar.write(f"Пропущено после 5 попыток: '{title}'")

                    time.sleep(1)

        print(f"\nРасчет завершён!")

In [52]:
extract_features()

Уже обработано статей: 6918
Найдено недостающих/ошибочных: 9
Обработка недостающих через Mistral...



Обработка статей: 100%|██████████| 9/9 [03:13<00:00, 21.51s/стран]


Расчет завершён!


### Конвертация результата в csv

In [49]:
def json_to_csv():
    if CSV_FILE.exists():
        log(f"{CSV_FILE.name} уже существует")
        return

    if not FINAL_FILE.exists():
        log("Нет финального json")
        return

    log("Создание CSV...")

    with open(FINAL_FILE, "r", encoding="utf-8") as infile, \
         open(CSV_FILE, "w", newline="", encoding="utf-8") as csvfile:

        fieldnames = ["title", "country", "city", "description", "attractions", "tips_for_traveler", "url"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        total = sum(1 for _ in open(FINAL_FILE, "r", encoding="utf-8"))
        for i, line in enumerate(infile):
            data = json.loads(line.strip())

            attractions_str = "; ".join(data.get("attractions") or [])

            row = {
                "title": data.get("title", ""),
                "country": data.get("country", ""),
                "city": data.get("city", ""),
                "description": data.get("description", ""),
                "attractions": attractions_str,
                "tips_for_traveler": data.get("tips_for_traveler", ""),
                "url": data.get("url", "")
            }
            writer.writerow(row)

            if (i + 1) % 200 == 0 or (i + 1) == total:
                log(f"Обработано {i + 1}/{total}")

    log(f"Итоговый CSV: {CSV_FILE}")

In [54]:
json_to_csv()

[*] Создание CSV...
[*] Обработано 200/6919
[*] Обработано 400/6919
[*] Обработано 600/6919
[*] Обработано 800/6919
[*] Обработано 1000/6919
[*] Обработано 1200/6919
[*] Обработано 1400/6919
[*] Обработано 1600/6919
[*] Обработано 1800/6919
[*] Обработано 2000/6919
[*] Обработано 2200/6919
[*] Обработано 2400/6919
[*] Обработано 2600/6919
[*] Обработано 2800/6919
[*] Обработано 3000/6919
[*] Обработано 3200/6919
[*] Обработано 3400/6919
[*] Обработано 3600/6919
[*] Обработано 3800/6919
[*] Обработано 4000/6919
[*] Обработано 4200/6919
[*] Обработано 4400/6919
[*] Обработано 4600/6919
[*] Обработано 4800/6919
[*] Обработано 5000/6919
[*] Обработано 5200/6919
[*] Обработано 5400/6919
[*] Обработано 5600/6919
[*] Обработано 5800/6919
[*] Обработано 6000/6919
[*] Обработано 6200/6919
[*] Обработано 6400/6919
[*] Обработано 6600/6919
[*] Обработано 6800/6919
[*] Обработано 6919/6919
[*] Итоговый CSV: dataset/wikivoyage_ru_final.csv


In [51]:
log("Данные находятся в папке dataset:")
log("wikivoyage_ru_final.jsonl")
log("wikivoyage_ru_final.csv")

[*] Данные находятся в папке dataset:
[*] wikivoyage_ru_final.jsonl
[*] wikivoyage_ru_final.csv
